In [4]:
# @title Setup
from google.cloud import bigquery
from google.colab import data_table
import bigframes.pandas as bpd
import pandas as pd

project = 'green-taxi-trips-analytics' # Project ID inserted based on the query results selected to explore
location = 'US' # Location inserted based on the query results selected to explore
client = bigquery.Client(project=project, location=location)
data_table.enable_dataframe_formatter()

In [5]:
# Function to execute a BigQuery query and return a DataFrame

def query_to_dataframe(query: str) -> pd.DataFrame:
    """
    Executes a SQL query in BigQuery and returns a Pandas DataFrame.

    Parameters:
    - query (str): The SQL query to execute.

    Return:
    - pd.DataFrame : The DataFrame containing the results of the query.
    """
    try:
        df = client.query(query).to_dataframe()
        print(f"Query executed successfully. Retrieved {df.shape[0]} rows.")
        return df
    except Exception as e:
        print(f"Error executing query: {e}")
        return pd.DataFrame()

In [10]:
query_trips_ml_data = """
SELECT
  * EXCEPT(ehail_fee)  -- Exclut uniquement la colonne ehail_fee
FROM
  `green-taxi-trips-analytics.ml_dataset.trips_ml_data`
"""
trips_ml_data_df = query_to_dataframe(query_trips_ml_data)
trips_ml_data_df.head()

Query executed successfully. Retrieved 634558 rows.


,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge,source_file
0,1,2024-10-26 05:10:45+00:00,2024-10-26 05:13:46+00:00,N,5.0,82,82,1.0,3.10,0.0,0.00,0.5,0.0,0.0,0.0,0.50,2.0,2.0,0.0,green_tripdata_2024-10.parquet
1,1,2024-09-26 11:54:17+00:00,2024-09-26 12:00:35+00:00,N,5.0,210,210,1.0,0.10,0.0,0.00,0.5,0.0,0.0,0.0,0.50,2.0,2.0,0.0,green_tripdata_2024-09.parquet
2,1,2025-01-29 18:34:07+00:00,2025-01-29 18:52:28+00:00,N,5.0,75,50,1.0,4.40,0.0,0.75,0.0,0.0,0.0,0.0,0.75,2.0,2.0,0.0,green_tripdata_2025-01.parquet
3,1,2025-01-25 14:40:36+00:00,2025-01-25 14:57:33+00:00,N,5.0,196,148,1.0,8.20,0.0,0.75,0.0,0.0,0.0,0.0,0.75,2.0,2.0,0.0,green_tripdata_2025-01.parquet
4,2,2024-12-31 13:41:32+00:00,2024-12-31 13:47:38+00:00,N,5.0,74,42,2.0,1.03,0.0,0.00,0.0,0.0,0.0,1.0,1.00,2.0,2.0,0.0,green_tripdata_2024-12.parquet


In [11]:
trips_ml_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 634558 entries, 0 to 634557
Data columns (total 20 columns):
 #   Column                 Non-Null Count   Dtype              
---  ------                 --------------   -----              
 0   VendorID               634558 non-null  Int64              
 1   lpep_pickup_datetime   634558 non-null  datetime64[us, UTC]
 2   lpep_dropoff_datetime  634558 non-null  datetime64[us, UTC]
 3   store_and_fwd_flag     634558 non-null  object             
 4   RatecodeID             634558 non-null  float64            
 5   PULocationID           634558 non-null  Int64              
 6   DOLocationID           634558 non-null  Int64              
 7   passenger_count        634558 non-null  float64            
 8   trip_distance          634558 non-null  float64            
 9   fare_amount            634558 non-null  float64            
 10  extra                  634558 non-null  float64            
 11  mta_tax                634558 non-null 

In [12]:
# Missing values
trips_ml_data_df.isna().sum()

,0
VendorID,0
lpep_pickup_datetime,0
lpep_dropoff_datetime,0
store_and_fwd_flag,0
RatecodeID,0
PULocationID,0
DOLocationID,0
passenger_count,0
trip_distance,0
fare_amount,0


In [14]:
def preprocess_data(df):
    # Ensure datetime columns are in datetime format
    #df["lpep_pickup_datetime"] = pd.to_datetime(df["lpep_pickup_datetime"])
    #df["lpep_dropoff_datetime"] = pd.to_datetime(df["lpep_dropoff_datetime"])

    # Trip duration in minutes
    df["trip_duration"] = (df["lpep_dropoff_datetime"] - df["lpep_pickup_datetime"]).dt.total_seconds() / 60

    # Extract time-based features
    df["pickup_dayofweek"] = df["lpep_pickup_datetime"].dt.dayofweek # Monday=0, Sunday=6.
    df["pickup_month"] = df["lpep_pickup_datetime"].dt.month
    df["pickup_year"] = df["lpep_pickup_datetime"].dt.year
    df["pickup_hour"] = df["lpep_pickup_datetime"].dt.hour
    df["is_weekend"] = df["pickup_dayofweek"].isin([5, 6]).astype(int)  # 5=Saturday, 6=Sunday

    # Filter payment type (Credit Card = 1, Cash = 2)
    #df = df[df["payment_type"].isin([1, 2])].copy()

    # Create binary feature for credit card payments
    df["is_credit_card"] = (df["payment_type"] == 1).astype(int)

    # Select relevant columns
    selected_cols = [
        "PULocationID", "DOLocationID", "passenger_count", "trip_distance",
        "trip_duration", "pickup_dayofweek", "pickup_month", "pickup_year", "pickup_hour",
        "is_weekend", "is_credit_card", "total_amount"
    ]

    return df[selected_cols].copy()


In [15]:
from sklearn.model_selection import train_test_split

def split_data(df, train_size=0.7, val_size=0.15, test_size=0.15, random_state=42):
    """
    Splits the dataframe into train, validation, and test sets.

    Parameters:
    - df: Pandas DataFrame
    - train_size: Proportion of the dataset for training (default=70%)
    - val_size: Proportion for validation (default=15%)
    - test_size: Proportion for testing (default=15%)
    - random_state: Seed for reproducibility

    Returns:
    - train_df, val_df, test_df: Split DataFrames
    """
    assert train_size + val_size + test_size == 1, "Split sizes must sum to 1"

    # First, split train + val and test
    train_val_df, test_df = train_test_split(df, test_size=test_size, random_state=random_state)

    # Then, split train and validation
    train_df, val_df = train_test_split(train_val_df, test_size=val_size / (train_size + val_size),
                                        random_state=random_state)

    return train_df, val_df, test_df

# Apply the function
train_df, val_df, test_df = split_data(trips_ml_data_df)

# Display the sizes
print(f"Train size: {len(train_df)}")
print(f"Validation size: {len(val_df)}")
print(f"Test size: {len(test_df)}")


Train size: 444190
Validation size: 95184
Test size: 95184


In [16]:
preprocessed_train_df = preprocess_data(train_df)
preprocessed_train_df.head()

,PULocationID,DOLocationID,passenger_count,trip_distance,trip_duration,pickup_dayofweek,pickup_month,pickup_year,pickup_hour,is_weekend,is_credit_card,total_amount
210721,74,75,1.0,1.70,8.916667,4,2,2024,18,0,1,16.20
54298,74,263,1.0,1.15,4.100000,3,3,2024,6,0,1,9.70
300878,74,236,1.0,1.74,11.816667,1,11,2024,16,0,1,23.46
25328,223,223,5.0,0.78,3.866667,2,11,2024,22,0,1,9.96
2052,65,97,2.0,0.16,0.766667,6,6,2024,19,1,0,5.20


In [17]:
preprocessed_train_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 444190 entries, 210721 to 224372
Data columns (total 12 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   PULocationID      444190 non-null  Int64  
 1   DOLocationID      444190 non-null  Int64  
 2   passenger_count   444190 non-null  float64
 3   trip_distance     444190 non-null  float64
 4   trip_duration     444190 non-null  float64
 5   pickup_dayofweek  444190 non-null  int32  
 6   pickup_month      444190 non-null  int32  
 7   pickup_year       444190 non-null  int32  
 8   pickup_hour       444190 non-null  int32  
 9   is_weekend        444190 non-null  int64  
 10  is_credit_card    444190 non-null  int64  
 11  total_amount      444190 non-null  float64
dtypes: Int64(2), float64(4), int32(4), int64(2)
memory usage: 38.1 MB


In [18]:
# Load the preprocessed_train_df dataframe into BigQuery

DATASET_ID = "ml_dataset"
TABLE_ID = "preprocessed_train_data"
FULL_TABLE_ID = f"{project}.{DATASET_ID}.{TABLE_ID}"

# Define schema (ensure correct types)
schema = [
    bigquery.SchemaField("PULocationID", "INTEGER"),
    bigquery.SchemaField("DOLocationID", "INTEGER"),
    bigquery.SchemaField("passenger_count", "FLOAT"),
    bigquery.SchemaField("trip_distance", "FLOAT"),
    bigquery.SchemaField("trip_duration", "FLOAT"),
    bigquery.SchemaField("pickup_dayofweek", "INTEGER"),
    bigquery.SchemaField("pickup_month", "INTEGER"),
    bigquery.SchemaField("pickup_year", "INTEGER"),
    bigquery.SchemaField("pickup_hour", "INTEGER"),
    bigquery.SchemaField("is_weekend", "INTEGER"),
    bigquery.SchemaField("is_credit_card", "INTEGER"),
    bigquery.SchemaField("total_amount", "FLOAT"),
]

# Load data into BigQuery
job = client.load_table_from_dataframe(
    preprocessed_train_df, FULL_TABLE_ID, job_config=bigquery.LoadJobConfig(schema=schema)
)

# Wait for the job to complete
job.result()

print(f"Data successfully uploaded to BigQuery: {FULL_TABLE_ID}")

Data successfully uploaded to BigQuery: green-taxi-trips-analytics.ml_dataset.preprocessed_train_data


In [19]:
preprocessed_test_df = preprocess_data(test_df)
preprocessed_test_df.head()

,PULocationID,DOLocationID,passenger_count,trip_distance,trip_duration,pickup_dayofweek,pickup_month,pickup_year,pickup_hour,is_weekend,is_credit_card,total_amount
502919,74,239,3.0,2.92,23.033333,3,7,2024,9,0,1,31.38
181477,256,112,2.0,1.68,7.866667,6,7,2024,2,1,1,15.00
20526,130,130,1.0,0.39,4.133333,5,3,2024,20,1,0,8.30
550741,75,229,1.0,2.86,33.000000,1,12,2024,15,0,1,34.95
490804,43,163,1.0,2.34,25.933333,1,8,2024,14,0,0,25.45


In [20]:
preprocessed_test_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 95184 entries, 502919 to 232350
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   PULocationID      95184 non-null  Int64  
 1   DOLocationID      95184 non-null  Int64  
 2   passenger_count   95184 non-null  float64
 3   trip_distance     95184 non-null  float64
 4   trip_duration     95184 non-null  float64
 5   pickup_dayofweek  95184 non-null  int32  
 6   pickup_month      95184 non-null  int32  
 7   pickup_year       95184 non-null  int32  
 8   pickup_hour       95184 non-null  int32  
 9   is_weekend        95184 non-null  int64  
 10  is_credit_card    95184 non-null  int64  
 11  total_amount      95184 non-null  float64
dtypes: Int64(2), float64(4), int32(4), int64(2)
memory usage: 8.2 MB


In [21]:
preprocessed_test_df.shape

(95184, 12)

In [22]:
# Load the preprocessed_test_df dataframe into BigQuery

DATASET_ID = "ml_dataset"
TABLE_ID = "preprocessed_test_data"
FULL_TABLE_ID = f"{project}.{DATASET_ID}.{TABLE_ID}"

# Define schema (ensure correct types)
schema = [
    bigquery.SchemaField("PULocationID", "INTEGER"),
    bigquery.SchemaField("DOLocationID", "INTEGER"),
    bigquery.SchemaField("passenger_count", "FLOAT"),
    bigquery.SchemaField("trip_distance", "FLOAT"),
    bigquery.SchemaField("trip_duration", "FLOAT"),
    bigquery.SchemaField("pickup_dayofweek", "INTEGER"),
    bigquery.SchemaField("pickup_month", "INTEGER"),
    bigquery.SchemaField("pickup_year", "INTEGER"),
    bigquery.SchemaField("pickup_hour", "INTEGER"),
    bigquery.SchemaField("is_weekend", "INTEGER"),
    bigquery.SchemaField("is_credit_card", "INTEGER"),
    bigquery.SchemaField("total_amount", "FLOAT"),
]

# Load data into BigQuery
job = client.load_table_from_dataframe(
    preprocessed_test_df, FULL_TABLE_ID, job_config=bigquery.LoadJobConfig(schema=schema)
)

# Wait for the job to complete
job.result()

print(f"Data successfully uploaded to BigQuery: {FULL_TABLE_ID}")

Data successfully uploaded to BigQuery: green-taxi-trips-analytics.ml_dataset.preprocessed_test_data


In [ ]:
# You can continue to create a custom model
